### BBC

In [2]:
!pip install selenium

  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.0/9.7 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.0/9.7 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.0/9.7 MB 219.4 kB/s eta 0:00:45
   ---------------------------------------- 0.1/9.7 MB 297.7 kB/s eta 0:00:33
   ---------------------------------------- 0.1/9.7 MB 245.8 kB/s eta 0:00:40
   ---------------------------------------- 0.1/9.7 MB 245.8 kB/s eta 0:00:40
   ---------------------------------------- 0.1/9.7 MB 275.8 kB/s eta 0:00:35
   ---------------------------------------- 0.1/9.7 MB 297.7 kB/s eta 0:00:33
    --------------------------------------- 0.1/9.7 MB 288.8 kB/s eta 0:00:34
    --------------------------------------- 0.2/9.7 MB 316.5 kB/s eta 0:00:31
    -------------

In [1]:
import pandas as pd
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

# Path to Microsoft Edge WebDriver
msedgedriver_path = r'C:\\Users\\mouns\\Documents\\fiverr\\Orders\\Order 16\\scrapping tool\\msedgedriver.exe'
options = webdriver.EdgeOptions()

# Load company and crypto names
df = pd.read_excel('company_crypto_list.xlsx')  # Your input file with company/crypto names
names = df['Name'].dropna().tolist()  # Get the list of names

In [2]:
names

['Tesla']

In [ ]:
import pandas as pd
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
import time

# Path to Microsoft Edge WebDriver
msedgedriver_path = r'C:\\Users\\mouns\\Documents\\fiverr\\Orders\\Order 16\\scrapping tool\\msedgedriver.exe'
options = webdriver.EdgeOptions()
options.add_argument('--headless')  # Run in headless mode for better performance
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Load company and crypto names
df = pd.read_excel('company_crypto_list.xlsx')
names = df['Name'].dropna().tolist()

# Initialize a list to store article data
article_data = []


# Function to search, click the first link, scrape data, and close the browser
def search_and_scrape(name, news_site):
    try:
        service = Service(executable_path=msedgedriver_path)
        driver = webdriver.Edge(service=service, options=options)
        driver.set_page_load_timeout(15)  # Set a page load timeout
        search_query = f"{news_site} {name}"
        search_url = f"https://www.google.com/search?q={search_query}"

        driver.get(search_url)
        wait = WebDriverWait(driver, 10)

        # Locate the results and find "bbc.com" link
        results = driver.find_elements(By.CSS_SELECTOR, "div.g")
        found_bbc_link = False

        for result in results:
            try:
                cite_element = result.find_element(By.TAG_NAME, "cite")
                if "bbc.com" in cite_element.text:
                    link = result.find_element(By.TAG_NAME, "a").get_attribute("href")
                    print(f"Found BBC article for '{name}': {link}")
                    driver.get(link)
                    found_bbc_link = True
                    break
            except NoSuchElementException:
                continue

        if not found_bbc_link:
            print(f"No BBC link found for '{name}' on {news_site}.")
            return

        # Scrape data from the BBC article
        time.sleep(random.uniform(2, 4))  # Reduced sleep time

        try:
            title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#main-content > article > div:nth-child(1) > h1'))).text
        except TimeoutException:
            title = 'NA'

        try:
            author = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="byline-new"] .sc-2b5e3b35-7').text
        except NoSuchElementException:
            author = 'NA'

        try:
            date = driver.find_element(By.CSS_SELECTOR, 'div.sc-2b5e3b35-1.jTEdni time').text
        except NoSuchElementException:
            date = 'NA'

        paragraphs = driver.find_elements(By.CSS_SELECTOR, 'div[data-component="text-block"] p.sc-eb7bd5f6-0.fYAfXe')
        text = ' '.join([para.text for para in paragraphs])

        article_data.append({
            'Name': name,
            'Website': news_site,
            'Link': driver.current_url,
            'Title': title,
            'Author': author,
            'Date': date,
            'Text': text
        })

    except (TimeoutException, NoSuchElementException, WebDriverException) as e:
        print(f"Error for '{name}' on {news_site}: {e}")
    finally:
        driver.quit()

# Main function to process each name
def process_names(names):
    max_threads = min(5, len(names))  # Adjust thread count based on the number of names
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = [executor.submit(search_and_scrape, name, "BBC") for name in names]
        for future in as_completed(futures):
            try:
                future.result()  # Collect exceptions if any
            except Exception as e:
                print(f"An error occurred: {e}")

# Run the script
process_names(names)

# Create a DataFrame from the article data and save it to Excel
if article_data:
    df_articles = pd.DataFrame(article_data)
    df_articles.to_excel('bbc_articles.xlsx', index=False)
    print("Data saved to 'bbc_articles.xlsx'.")
else:
    print("No articles scraped.")

print("Google search and scrape process completed.")


Found BBC article for 'Tesla': https://www.bbc.com/news/topics/c8nq32jwjnmt
Data saved to 'bbc_articles.xlsx'.
Google search and scrape process completed.


In [21]:
import pandas as pd
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
import time

# Path to Microsoft Edge WebDriver
msedgedriver_path = r'C:\\Users\\mouns\\Documents\\fiverr\\Orders\\Order 16\\scrapping tool\\msedgedriver.exe'
options = webdriver.EdgeOptions()
options.add_argument('--headless')  # Run in headless mode for better performance
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

def scrap_articles_bbc(names_list):
    article_data = []

    # Function to search, click the first link, scrape data, and close the browser
    def search_and_scrape(name, news_site="BBC"):
        try:
            service = Service(executable_path=msedgedriver_path)
            driver = webdriver.Edge(service=service, options=options)
            driver.set_page_load_timeout(15)  # Set a page load timeout
            search_query = f"{news_site} {name}"
            search_url = f"https://www.google.com/search?q={search_query}"

            driver.get(search_url)
            wait = WebDriverWait(driver, 10)

            # Locate the results and find "bbc.com" link
            results = driver.find_elements(By.CSS_SELECTOR, "div.g")
            found_bbc_link = False

            for result in results:
                try:
                    cite_element = result.find_element(By.TAG_NAME, "cite")
                    if "bbc.com" in cite_element.text:
                        link = result.find_element(By.TAG_NAME, "a").get_attribute("href")
                        print(f"Found BBC article for '{name}': {link}")
                        driver.get(link)
                        found_bbc_link = True
                        break
                except NoSuchElementException:
                    continue

            if not found_bbc_link:
                print(f"No BBC link found for '{name}' on {news_site}.")
                return

            # Wait for the content to load and scrape data from the BBC article
            time.sleep(random.uniform(2, 4))

            try:
                title = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h1'))).text
            except TimeoutException:
                title = 'NA'

            try:
                author = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="byline"]').text
            except NoSuchElementException:
                author = 'NA'

            try:
                date = driver.find_element(By.CSS_SELECTOR, 'time').get_attribute('datetime')
            except NoSuchElementException:
                date = 'NA'

            paragraphs = driver.find_elements(By.CSS_SELECTOR, 'article p')
            text = ' '.join([para.text for para in paragraphs if para.text])

            article_data.append({
                'Name': name,
                'Website': news_site,
                'Link': driver.current_url,
                'Title': title,
                'Author': author,
                'Date': date,
                'Text': text
            })

        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            print(f"Error for '{name}' on {news_site}: {e}")
        finally:
            driver.quit()

    # Main function to process each name
    def process_names(names):
        max_threads = min(5, len(names))  # Adjust thread count based on the number of names
        with ThreadPoolExecutor(max_workers=max_threads) as executor:
            futures = [executor.submit(search_and_scrape, name) for name in names]
            for future in as_completed(futures):
                try:
                    future.result()  # Collect exceptions if any
                except Exception as e:
                    print(f"An error occurred: {e}")

    # Run the script
    process_names(names_list)

    # Return the collected article data in JSON format
    return json.dumps(article_data, ensure_ascii=False, indent=4)

# Example usage
names = ['Apple', 'Amazon', 'NVIDIA', 'Bitcoin']
result_json = scrap_articles_bbc(names)
print(result_json)


Found BBC article for 'Apple': https://www.bbc.com/news/topics/crr7mlg0gqqt
Found BBC article for 'Bitcoin': https://www.bbc.com/news/topics/c734j90em14t
Found BBC article for 'Amazon': https://www.bbc.com/news/topics/c50znx8v8y4t
Found BBC article for 'NVIDIA': https://www.bbc.com/news/topics/cewrlqz847yt
[
    {
        "Name": "Apple",
        "Website": "BBC",
        "Link": "https://www.bbc.com/news/topics/crr7mlg0gqqt",
        "Title": "Apple",
        "Author": "NA",
        "Date": "NA",
        "Text": "Google, Apple and TomTom say changes will be made to audio prompts after two women died. The EU ordered Apple to pay Ireland billions in unpaid taxes in September. It is the latest move from the tech giant to change how people search online using artificial intelligence. In July, Jack Chambers pledged €1.4bn in tax measures and new expenditure of €6.9bn It is the time of year when new handsets are unveiled, but they may offer only small improvements. Apple has been ordered to

In [18]:
import pandas as pd
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
import time

# Path to Microsoft Edge WebDriver
msedgedriver_path = r'C:\\Users\\mouns\\Documents\\fiverr\\Orders\\Order 16\\scrapping tool\\msedgedriver.exe'
options = webdriver.EdgeOptions()
options.add_argument('--headless')  # Run in headless mode for better performance
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


def scrap_articles_bbc(subject):

# Load company and crypto names
df = pd.read_excel('company_crypto_list.xlsx')
names = df['Name'].dropna().tolist()

# Initialize a list to store article data
article_data = []

# Function to search, click the first link, scrape data, and close the browser
def search_and_scrape(name, news_site):
    try:
        service = Service(executable_path=msedgedriver_path)
        driver = webdriver.Edge(service=service, options=options)
        driver.set_page_load_timeout(15)  # Set a page load timeout
        search_query = f"{news_site} {name}"
        search_url = f"https://www.google.com/search?q={search_query}"

        driver.get(search_url)
        wait = WebDriverWait(driver, 10)

        # Locate the results and find "bbc.com" link
        results = driver.find_elements(By.CSS_SELECTOR, "div.g")
        found_bbc_link = False

        for result in results:
            try:
                cite_element = result.find_element(By.TAG_NAME, "cite")
                if "bbc.com" in cite_element.text:
                    link = result.find_element(By.TAG_NAME, "a").get_attribute("href")
                    print(f"Found BBC article for '{name}': {link}")
                    driver.get(link)
                    found_bbc_link = True
                    break
            except NoSuchElementException:
                continue

        if not found_bbc_link:
            print(f"No BBC link found for '{name}' on {news_site}.")
            return

        # Scrape data from the BBC article
        time.sleep(random.uniform(2, 4))  # Reduced sleep time

        try:
            title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#main-content > article > div:nth-child(1) > h1'))).text
        except TimeoutException:
            title = 'NA'

        try:
            author = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="byline-new"] .sc-2b5e3b35-7').text
        except NoSuchElementException:
            author = 'NA'

        try:
            date = driver.find_element(By.CSS_SELECTOR, 'div.sc-2b5e3b35-1.jTEdni time').text
        except NoSuchElementException:
            date = 'NA'

        paragraphs = driver.find_elements(By.CSS_SELECTOR, 'div[data-component="text-block"] p.sc-eb7bd5f6-0.fYAfXe')
        text = ' '.join([para.text for para in paragraphs])

        article_data.append({
            'Name': name,
            'Website': news_site,
            'Link': driver.current_url,
            'Title': title,
            'Author': author,
            'Date': date,
            'Text': text
        })

    except (TimeoutException, NoSuchElementException, WebDriverException) as e:
        print(f"Error for '{name}' on {news_site}: {e}")
    finally:
        driver.quit()

# Main function to process each name
def process_names(names):
    max_threads = min(5, len(names))  # Adjust thread count based on the number of names
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = [executor.submit(search_and_scrape, name, "BBC") for name in names]
        for future in as_completed(futures):
            try:
                future.result()  # Collect exceptions if any
            except Exception as e:
                print(f"An error occurred: {e}")

# Run the script
process_names(names)

# Create a DataFrame from the article data and save it to Excel
if article_data:
    df_articles = pd.DataFrame(article_data)
    df_articles.to_excel('bbc_articles.xlsx', index=False)
    print("Data saved to 'bbc_articles.xlsx'.")
else:
    print("No articles scraped.")

print("Google search and scrape process completed.")


IndentationError: expected an indented block after function definition on line 21 (413102873.py, line 24)

In [7]:
import pandas as pd
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
import time
from django.http import JsonResponse

# Path to Microsoft Edge WebDriver
msedgedriver_path = r'C:\\Users\\mouns\\Documents\\fiverr\\Orders\\Order 16\\scrapping tool\\msedgedriver.exe'
options = webdriver.EdgeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Function to search and scrape articles
def search_and_scrape(name, news_site="BBC"):
    article_data = []
    try:
        service = Service(executable_path=msedgedriver_path)
        driver = webdriver.Edge(service=service, options=options)
        driver.set_page_load_timeout(15)
        search_query = f"{news_site} {name}"
        search_url = f"https://www.google.com/search?q={search_query}"

        driver.get(search_url)
        wait = WebDriverWait(driver, 10)

        # Locate the results and find "bbc.com" link
        results = driver.find_elements(By.CSS_SELECTOR, "div.g")
        found_bbc_link = False

        for result in results:
            try:
                cite_element = result.find_element(By.TAG_NAME, "cite")
                if "bbc.com" in cite_element.text:
                    link = result.find_element(By.TAG_NAME, "a").get_attribute("href")
                    driver.get(link)
                    found_bbc_link = True
                    break
            except NoSuchElementException:
                continue

        if not found_bbc_link:
            return {"error": f"No BBC link found for '{name}' on {news_site}."}

        # Scrape data from the BBC article
        time.sleep(random.uniform(2, 4))

        try:
            title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#main-content > article > div:nth-child(1) > h1'))).text
        except TimeoutException:
            title = 'NA'

        try:
            author = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="byline-new"] .sc-2b5e3b35-7').text
        except NoSuchElementException:
            author = 'NA'

        try:
            date = driver.find_element(By.CSS_SELECTOR, 'div.sc-2b5e3b35-1.jTEdni time').text
        except NoSuchElementException:
            date = 'NA'

        paragraphs = driver.find_elements(By.CSS_SELECTOR, 'div[data-component="text-block"] p.sc-eb7bd5f6-0.fYAfXe')
        text = ' '.join([para.text for para in paragraphs])

        article_data.append({
            'Name': name,
            'Website': news_site,
            'Link': driver.current_url,
            'Title': title,
            'Author': author,
            'Date': date,
            'Text': text
        })

    except (TimeoutException, NoSuchElementException, WebDriverException) as e:
        return {"error": f"Error for '{name}' on {news_site}: {e}"}
    finally:
        driver.quit()

    return article_data

# Django view function
def scrape_articles_view(request):
    if request.method == 'POST':
        names = request.POST.getlist('names')
        if not names:
            return JsonResponse({"error": "No names provided"}, status=400)

        max_threads = min(5, len(names))
        results = []
        with ThreadPoolExecutor(max_workers=max_threads) as executor:
            futures = [executor.submit(search_and_scrape, name) for name in names]
            for future in as_completed(futures):
                result = future.result()
                if isinstance(result, list):
                    results.extend(result)
                else:
                    results.append(result)

        return JsonResponse(results, safe=False)
    else:
        return JsonResponse({"error": "Invalid request method. Use POST."}, status=405)

ModuleNotFoundError: No module named 'django'

In [17]:
import random
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
import time

# Path to Microsoft Edge WebDriver
msedgedriver_path = r'C:\\Users\\mouns\\Documents\\fiverr\\Orders\\Order 16\\scrapping tool\\msedgedriver.exe'
options = webdriver.EdgeOptions()
options.add_argument('--headless')  # Run in headless mode for better performance
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

def search_and_scrape(name, news_site="BBC"):
    article_data = []
    try:
        service = Service(executable_path=msedgedriver_path)
        driver = webdriver.Edge(service=service, options=options)
        driver.set_page_load_timeout(15)
        search_query = f"{news_site} {name}"
        search_url = f"https://www.google.com/search?q={search_query}"

        driver.get(search_url)
        wait = WebDriverWait(driver, 15)

        # Collect articles across multiple pages
        bbc_links = []

        while len(bbc_links) < 3:  # Stop after collecting at least 3 articles
            results = driver.find_elements(By.CSS_SELECTOR, "div.g")
            for result in results:
                try:
                    cite_element = result.find_element(By.TAG_NAME, "cite")
                    if "bbc.com" in cite_element.text:
                        link = result.find_element(By.TAG_NAME, "a").get_attribute("href")
                        if link not in bbc_links:
                            bbc_links.append(link)
                            print(f"Found BBC article for '{name}': {link}")
                            if len(bbc_links) >= 3:
                                break
                except NoSuchElementException:
                    continue

            # Check if there are more pages and click "Next" if available
            try:
                next_button = driver.find_element(By.ID, "pnnext")
                next_button.click()
                time.sleep(random.uniform(2, 4))  # Random delay for loading the next page
            except NoSuchElementException:
                print("No more pages available.")
                break

        if not bbc_links:
            return {"error": f"No BBC link found for '{name}' on {news_site}."}

        # Iterate through the collected links and scrape them
        for link in bbc_links[:3]:  # Scrape only the first 3 unique links collected
            driver.get(link)
            time.sleep(random.uniform(2, 4))  # Random delay for page load

            # Extract title
            try:
                title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1'))).text
            except TimeoutException:
                title = 'NA'

            # Extract author (try multiple selectors)
            try:
                author = driver.find_element(By.CSS_SELECTOR, 'div[data-testid*="byline"]').text
            except NoSuchElementException:
                try:
                    author = driver.find_element(By.CSS_SELECTOR, '.byline__name').text  # Alternative selector
                except NoSuchElementException:
                    author = 'NA'

            # Extract date (try multiple selectors)
            try:
                date = driver.find_element(By.CSS_SELECTOR, 'time').text
            except NoSuchElementException:
                try:
                    date = driver.find_element(By.CSS_SELECTOR, 'span.date').text  # Alternative selector
                except NoSuchElementException:
                    date = 'NA'

            # Extract text
            paragraphs = driver.find_elements(By.CSS_SELECTOR, 'article p')
            text = ' '.join([para.text for para in paragraphs])

            article_data.append({
                'Name': name,
                'Website': news_site,
                'Link': driver.current_url,
                'Title': title,
                'Author': author,
                'Date': date,
                'Text': text
            })

    except (TimeoutException, NoSuchElementException, WebDriverException) as e:
        print(f"Error for '{name}' on {news_site}: {e}")
    finally:
        driver.quit()

    return article_data


result = search_and_scrape("Trump")
print("hhhhh")
print(result)


Found BBC article for 'Trump': https://www.bbc.com/news/topics/cp7r8vgl2lgt
Found BBC article for 'Trump': https://www.bbc.com/news/live/czxrnw5qrprt
Found BBC article for 'Trump': https://www.bbc.com/reel/video/p0k2snny/how-the-fall-of-the-berlin-wall-inspired-filmmakers
hhhhh
[{'Name': 'Trump', 'Website': 'BBC', 'Link': 'https://www.bbc.com/news/topics/cp7r8vgl2lgt', 'Title': '', 'Author': 'NA', 'Date': 'NA', 'Text': 'Republican Donald Trump, 78, is set to return to the White House after winning the 2024 presidential election against Kamala Harris. It marks a stunning political comeback for Trump four years after he lost the White House. Republican Donald Trump will be the next US president after securing a second term in the White House. Millions of Americans voted to give him a second chance - our North America editor Sarah Smith looks at why. Here\'s what you need to know about the Trump family. Darren Jones says there "shouldn\'t be an element of conceding to illegal invasions fr

[{'Name': 'Tesla', 'Website': 'BBC', 'Link': 'https://www.bbc.com/news/topics/c8nq32jwjnmt', 'Title': 'NA', 'Author': 'NA', 'Date': 'NA', 'Text': ''}]


#### NCC

Starting search for 'Tesla' on CNN
No CNN links found for 'Tesla'. Exiting search.
Final save: 0 articles saved to 'cnn_articles.xlsx'.


Starting search for 'Tesla' on CNN
Loading article page: https://www.cnn.com/markets/stocks/TSLA
Article title: Tesla, Inc.
Loading article page: https://www.cnn.com/2024/10/23/business/tesla-earnings/index.html
Article title: Tesla cuts its costs to build cars, boosting earnings
Loading article page: https://www.cnn.com/2024/10/18/business/tesla-fsd-federal-investigation/index.html
Article title: Feds investigating safety of Tesla’s ‘Full Self-Driving’ feature
Loading article page: https://www.cnn.com/2024/10/10/business/elon-musk-tesla-robotaxi-event/index.html
Article title: Elon Musk is promising to unveil the future of Tesla tonight
Loading article page: https://www.cnn.com/2024/10/10/business/elon-musk-tesla-robotaxis/index.html
Article title: Elon Musk unveils his ‘Cybercab’ robotaxi
Loading article page: https://www.cnn.com/2024/10/02/business/tesla-sales/index.html
Article title: Tesla posts first sales gain of the year but shares fall as competition heats up
Loading article p

Starting search for 'Tesla' on CNN
Loading article page: https://www.cnn.com/markets/stocks/TSLA
Article title: Tesla, Inc.
Loading article page: https://www.cnn.com/2024/10/23/business/tesla-earnings/index.html
Article title: Tesla cuts its costs to build cars, boosting earnings
Loading article page: https://www.cnn.com/2024/11/06/business/video/musk-net-worth-tesla-trump-digvid
Article title: 
Loading article page: https://www.cnn.com/2024/10/18/business/tesla-fsd-federal-investigation/index.html
Article title: Feds investigating safety of Tesla’s ‘Full Self-Driving’ feature
Loading article page: https://www.cnn.com/2024/10/10/business/elon-musk-tesla-robotaxi-event/index.html
Article title: Elon Musk is promising to unveil the future of Tesla tonight
Loading article page: https://www.cnn.com/2024/10/10/business/elon-musk-tesla-robotaxis/index.html
Article title: Elon Musk unveils his ‘Cybercab’ robotaxi
Loading article page: https://www.cnn.com/2024/10/02/business/tesla-sales/index.

In [8]:
import pandas as pd
import random
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Path to Microsoft Edge WebDriver
msedgedriver_path = r'C:\\Users\\mouns\\Documents\\fiverr\\Orders\\Order 16\\scrapping tool\\msedgedriver.exe'

# List of user agents to randomly choose from
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:86.0) Gecko/20100101 Firefox/86.0",
    "Mozilla/5.0 (Linux; Android 10; Pixel 3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.93 Mobile Safari/537.36"
]

# Load company names
df = pd.read_excel('company_crypto_list.xlsx')  # Your input file with company/crypto names
names = df['Name'].dropna().tolist()  # Get the list of names

# Initialize a list to store article data
article_data = []

# Function to save data to Excel
def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)

# Function to perform a random search to mimic human activity
def perform_random_search(driver):
    random_query = f"random search {random.randint(1000, 9999)}"
    search_url = f"https://www.google.com/search?q={random_query}"
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))  # Short delay to simulate browsing

# Function to search for CNN links and scrape data
def search_and_scrape(name):
    service = Service(executable_path=msedgedriver_path)

    def initialize_driver():
        options = Options()
        options.add_argument(f"user-agent={random.choice(user_agents)}")
        return webdriver.Edge(service=service, options=options)
    
    driver = initialize_driver()
    wait = WebDriverWait(driver, 15)
    
    search_query = f"{name} site:cnn.com"
    search_url = f"https://www.google.com/search?q={search_query}"
    
    try:
        print(f"Starting search for '{name}' on CNN")
        driver.get(search_url)
        
        # Random delay to avoid detection
        time.sleep(random.uniform(8, 12))
        
        # Locate and retrieve CNN links from search results
        cnn_links = []
        results = driver.find_elements(By.CSS_SELECTOR, 'div.g')
        for result in results:
            try:
                cite_element = result.find_element(By.TAG_NAME, 'cite')
                if "cnn.com" in cite_element.text:
                    link = result.find_element(By.TAG_NAME, 'a').get_attribute('href')
                    if link not in cnn_links:
                        cnn_links.append(link)
            except NoSuchElementException:
                continue
        
        if not cnn_links:
            print(f"No CNN links found for '{name}'. Exiting search.")
            return
        
        for link in cnn_links:
            # Perform a random search to avoid detection
            perform_random_search(driver)
            
            # Go to the CNN article link
            driver.get(link)
            time.sleep(random.uniform(8, 12))  # Random delay
            
            print(f"Loading article page: {link}")
            
            try:
                # Extract article title
                title_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1')))
                article_title = title_element.text
                print(f"Article title: {article_title}")
                
                # Use the current date as the article date
                current_date = datetime.now().strftime('%Y-%m-%d')
                
                # Extract Article Text
                try:
                    # Target paragraphs within the main article content container
                    text_elements = driver.find_elements(By.CSS_SELECTOR, 'div.article__content p.paragraph')
                    article_text = ' '.join([element.text for element in text_elements if element.text])
                except NoSuchElementException:
                    article_text = 'NA'
                
                # Store article data
                if article_text.strip():  # Ensures text is not just whitespace
                    article_data.append({
                        'Name': name,
                        'Article Title': article_title,
                        'Date': current_date,
                        'Text': article_text,
                        'Link': link
                    })

                    # Save to Excel after every 10 articles
                    if len(article_data) % 10 == 0:
                        save_to_excel(article_data, 'cnn_articles.xlsx')
                        print(f"Saved {len(article_data)} articles to 'cnn_articles.xlsx'.")
                else:
                    print(f"Skipped saving article from '{link}' because text is empty.")
            
            except Exception as e:
                print(f"Error extracting details from '{link}': {e}")
    
    except Exception as e:
        print(f"Error while processing '{name}': {e}")
    finally:
        driver.quit()

# Using ThreadPoolExecutor to process multiple names concurrently
with ThreadPoolExecutor(max_workers=2) as executor:
    futures = {executor.submit(search_and_scrape, name): name for name in names}
    for future in as_completed(futures):
        name = futures[future]
        try:
            future.result()
        except Exception as e:
            print(f"Error processing '{name}': {e}")

# Final save to ensure any remaining articles are saved
save_to_excel(article_data, 'cnn_articles.xlsx')
print(f"Final save: {len(article_data)} articles saved to 'cnn_articles.xlsx'.")


Starting search for 'Tesla' on CNN
Loading article page: https://www.cnn.com/markets/stocks/TSLA
Article title: Tesla, Inc.
Skipped saving article from 'https://www.cnn.com/markets/stocks/TSLA' because text is empty.
Loading article page: https://www.cnn.com/2024/10/23/business/tesla-earnings/index.html
Article title: Tesla cuts its costs to build cars, boosting earnings
Loading article page: https://www.cnn.com/2024/11/06/business/video/musk-net-worth-tesla-trump-digvid
Article title: 
Skipped saving article from 'https://www.cnn.com/2024/11/06/business/video/musk-net-worth-tesla-trump-digvid' because text is empty.
Loading article page: https://www.cnn.com/2024/10/18/business/tesla-fsd-federal-investigation/index.html
Article title: Feds investigating safety of Tesla’s ‘Full Self-Driving’ feature
Loading article page: https://www.cnn.com/2024/10/10/business/elon-musk-tesla-robotaxi-event/index.html
Article title: Elon Musk is promising to unveil the future of Tesla tonight
Loading ar